https://adventofcode.com/2023/day/12

In [1]:
import re

from collections import deque
from functools import cache

In [2]:
with open("data/12.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1"""

In [4]:
xlate = {ord("?"): "x", ord("."): "g", ord("#"): "b"}

In [5]:
def parse_input(data):
    L = []
    for line in data.splitlines():
        s, countstr = line.split()
        L.append(
            (
                s.translate(xlate),
                tuple(int(x) for x in countstr.split(",")),
            )
        )
    return L

In [6]:
parse_input(testdata)

[('xxxgbbb', (1, 1, 3)),
 ('gxxggxxgggxbbg', (1, 1, 3)),
 ('xbxbxbxbxbxbxbx', (1, 3, 1, 6)),
 ('xxxxgbgggbggg', (4, 1, 1)),
 ('xxxxgbbbbbbggbbbbbg', (1, 6, 5)),
 ('xbbbxxxxxxxx', (3, 2, 1))]

In [7]:
@cache
def make_res(bcounts):
    pat = "[^b]*" + "[^b]+".join(f"[bx]{{{n}}}" for n in bcounts) + "(?!=b)[^b]*$"
    return re.compile(pat)

In [8]:
# def make_res(bcounts):
#     pat = "[^b]*" + "[^b]+".join(f"(?<!b)[bx]{{{n}}}(?!=b)" for n in bcounts) + "[^b]*$"
#     return re.compile(pat)

In [9]:
@cache
def count_the_ways(s, bcounts, debug=False):
    bsre = make_res(bcounts)
    q = deque([s])
    ways = 0
    while q:
        s = q.popleft()
        if "x" not in s:
            m = bsre.match(s)
            if m:
                ways += 1
                if debug:
                    print(m.group())
            continue
        if bsre.match(s):
            q.append(s.replace("x", "g", 1))
            q.append(s.replace("x", "b", 1))
    return ways

In [10]:
def normalize(s):
    return re.sub(r"g+", "g", s)

In [11]:
count_the_ways("gxxxxbxgxxx", (1, 3, 3), debug=True)

ggbgbbbgbbb
gbggbbbgbbb
gbgbbbggbbb


3

In [12]:
%%time
for s, bcounts in parse_input(testdata):
    ways = count_the_ways(normalize(s), bcounts)
    print(s, bcounts, ways)

xxxgbbb (1, 1, 3) 1
gxxggxxgggxbbg (1, 1, 3) 4
xbxbxbxbxbxbxbx (1, 3, 1, 6) 1
xxxxgbgggbggg (4, 1, 1) 1
xxxxgbbbbbbggbbbbbg (1, 6, 5) 4
xbbbxxxxxxxx (3, 2, 1) 10
CPU times: user 553 µs, sys: 0 ns, total: 553 µs
Wall time: 550 µs


In [13]:
%%time
sum(count_the_ways(normalize(s), bcounts) for (s, bcounts) in parse_input(testdata))

CPU times: user 45 µs, sys: 4 µs, total: 49 µs
Wall time: 52.9 µs


21

In [14]:
%%time
sum(count_the_ways(normalize(s), bcounts) for (s, bcounts) in parse_input(data))

CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 126 ms


7173

In [15]:
# L = [(s, bcounts, count_the_ways(s, bcounts)) for (s, bcounts) in parse_input(data)]
# len(L)

In [16]:
# import pickle
# with open("goodhs.pickle", "wb") as f:
#     pickle.dump(L, f)

### Part 2

In [17]:
def unfold(s, bcounts, folds=5):
    return "x".join([s] * folds), bcounts * folds

In [18]:
unfold("???.###".translate(xlate), (1, 1, 3))

('xxxgbbbxxxxgbbbxxxxgbbbxxxxgbbbxxxxgbbb',
 (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3))

In [19]:
%%time
for s, bcounts in parse_input(testdata):
    s = normalize(s)
    print(s, bcounts, end=" ")
    for uf in range(1, 6):
        ways = count_the_ways(*unfold(s, bcounts, uf))
        print(ways, end=" ")
    print()

xxxgbbb (1, 1, 3) 1 1 1 1 1 
gxxgxxgxbbg (1, 1, 3) 4 32 256 2048 16384 
xbxbxbxbxbxbxbx (1, 3, 1, 6) 1 1 1 1 1 
xxxxgbgbg (4, 1, 1) 1 2 4 8 16 
xxxxgbbbbbbgbbbbbg (1, 6, 5) 4 20 100 500 2500 
xbbbxxxxxxxx (3, 2, 1) 10 150 2250 33750 506250 
CPU times: user 3.86 s, sys: 53.1 ms, total: 3.91 s
Wall time: 3.91 s


In [83]:
def fold5ways(f1, f2):
    r = f2 / f1
    d12 = f2 - f1*f1
    f3 = f1 * f2 + r * d12
    d23 = f3 - f1 * f2
    f4 = f3 * f1 + r * d23
    d34 = f4 - f3 * f1
    f5 = f4 * f1 + r * d34
    return f1, f2, f3, f4, f5


In [24]:
# %%time
# 3rd line of puzzle

# for folds in range(1, 6):
#     print(folds, count_the_ways(*unfold(".???.??.??".translate(xlate), (1,1,1), folds)))

1 16
2 775
3 44583
4 2732882
5 173257860
CPU times: user 16min 56s, sys: 24 ms, total: 16min 56s
Wall time: 16min 56s


In [84]:
fold5ways(16, 775)

(16, 775, 37539.0625, 1818298.33984375, 88073825.83618164)

Nope, no good

In [87]:
%%time
# 3rd line of puzzle

for folds in range(1, 5):
    print(folds, count_the_ways(*unfold(".???.??.??".translate(xlate), (1,1,1), folds)))

1 16
2 775
3 44583
4 2732882
CPU times: user 16.3 s, sys: 104 ms, total: 16.4 s
Wall time: 16.4 s
